## What a queue is
A queue is a data structure enforcing FIFO (first-in, first-out) ordering.
It supports:
- enqueue(x) → add to tail
- dequeue() → remove from head

## Why queues exist
Queues solve rate-mismatch problems between producers and consumers.
They preserve temporal order when processing cannot happen immediately.

## Operational behaviour
- FIFO ordering
- Constant-time head/tail operations
- Natural for scheduling, buffering, and event handling

## Variants
- Deque (double-ended queue)
- Priority queue
- Message queue
- Job/task queue

## Where queues appear
- Event loops
- OS schedulers
- Web server request buffers
- Messaging systems (Kafka, RabbitMQ)
- BFS graph traversal


In [2]:
# Basic FIFO queue
from collections import deque

q = deque()
q.append("task1")
q.append("task2")
q.append("task3")

print(q.popleft())  # task1
print(q.popleft())  # task2


task1
task2


In [11]:
# Queue implemented with two stacks
class Queue:
    def __init__(self):
        self.inbox = []
        self.outbox = []

    def enqueue(self, x):
        self.inbox.append(x)

    def dequeue(self):
        if not self.outbox:
            print(f"{self.inbox = }")
            while self.inbox:
                self.outbox.append(self.inbox.pop())
            print(f"{self.outbox = }")
        return self.outbox.pop()
    

q = Queue()

q.enqueue(1)
q.enqueue(2)
print(f"{q.dequeue()}")
q.enqueue(3)
q.enqueue(4)
print(f"{q.dequeue()}")
print(f"{q.dequeue()}")
print(f"{q.dequeue()}")


self.inbox = [1, 2]
self.outbox = [2, 1]
1
2
self.inbox = [3, 4]
self.outbox = [4, 3]
3
4


In [1]:
import asyncio
import random

async def producer(event_queue: asyncio.Queue):
    # enqueue some work then a sentinel to stop workers
    for i in range(3):
        await event_queue.put({"type": "print", "data": f"task {i}"})
        print(f"produced task {i}")
    await event_queue.put(None)
    await event_queue.put(None)  # sentinel to stop consumer

async def consumer(event_queue: asyncio.Queue):
    while True:
        event = await event_queue.get()
        if event is None:  
            print("shutdown signal")
            event_queue.task_done()
            break
        print(f"start {event['data']}")
        await asyncio.sleep(random.uniform(0.1, 0.5))  # simulate I/O delay
        print(f"end {event['data']}")
        event_queue.task_done()

async def main():
    event_queue = asyncio.Queue()
    prod = asyncio.create_task(producer(event_queue))
    cons1 = asyncio.create_task(consumer(event_queue))
    cons2 = asyncio.create_task(consumer(event_queue))
    await asyncio.gather(prod, cons1, cons2)

await main()

produced task 0
produced task 1
produced task 2
start task 0
start task 1
end task 0
start task 2
end task 1
shutdown signal
end task 2
shutdown signal


In [ ]:
import heapq

class PriorityQueue:
    def __init__(self):
        self._heap = []

    def push(self, priority, item):
        heapq.heappush(self._heap, (priority, item))

    def pop(self):
        return heapq.heappop(self._heap)

pq = PriorityQueue()
pq.push(10, "low")
pq.push(1, "urgent")
pq.push(5, "medium")
[pq.pop(), pq.pop(), pq.pop()]

In [ ]:
async def bounded_producer(q: asyncio.Queue, n=5):
    for i in range(n):
        await q.put(i)
        print(f"enqueued {i}, size={q.qsize()}")
    await q.put(None)

async def slow_consumer(q: asyncio.Queue):
    while True:
        item = await q.get()
        if item is None:
            q.task_done()
            break
        await asyncio.sleep(0.3)
        print(f"processed {item}")
        q.task_done()

async def demo_bounded():
    q = asyncio.Queue(maxsize=2)
    prod = asyncio.create_task(bounded_producer(q))
    cons = asyncio.create_task(slow_consumer(q))
    await asyncio.gather(prod, cons)

# await demo_bounded()  # uncomment to run

Notes:
- Priority queue uses heap; smallest priority first.
- Bounded queues force producers to backpressure when full; great for stabilizing throughput.